In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier

%run ../pyfiles/data_cleaning.py

In [5]:
# Set global random seed
np.random.seed(123)

### Import data and drop redundant data (rates)

In [6]:
# import data
df = pd.read_csv('../../data/deepsolar_tract.csv', encoding = 'latin-1')
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [7]:
# fips is a unique identifier
# Set fips as index 

df.set_index(keys = 'fips', inplace = True)
df.shape

(72537, 167)

In [8]:
cols = df.columns

In [9]:
df = drop_redundant_columns(df)

In [10]:
# Create our target column 'has_tiles', and drop additional redundant columns

df = create_has_tiles_target_column(df)

In [2]:
# # Figure out which variables are highly correlated, remove the most correlated ones one by one

# corr = pd.DataFrame((df.corr() > 0.8).sum())
# corr.sort_values(by = 0, ascending = False)[0:5]

In [77]:
# # Add highly correlated variables to list 'to_drop'
# to_drop = ['poverty_family_count','education_population','population', 'household_count','housing_unit_occupied_count', 'electricity_price_overall']

In [1]:
# # Drop highly colinear variables
# df = df.drop(to_drop, axis = 1)

### Checking for missing values

In [80]:
nulls = pd.DataFrame(df.isna().sum())
nulls.columns = ["missing"]
nulls[nulls['missing']>0].head()

,missing
average_household_income,886
gini_index,847
land_area,24
per_capita_income,643
population_density,316


In [11]:
# drop all missing values
df = df.dropna(axis = 0)

### Add more data on opportunity zones

### Train test split

### SMOTE!

### Scale Data 

### Steps for Modeling


Gridsearch:
Try -  All our variables in logisitic regression
 - params: C and penalty
 
Gridsearch:
Try - All variables in decision trees
 - params: max_depth (5, 8, 15, 25, 30), min_samples_split (1, 2, 5, 10, 15, 100), min_samples_leaf (1, 2, 5, 10)
 - max_depth : how deep the tree is
 - min_samples_split: minimum number of samples required to split an internal node
 - min_samples_leaf: he minimum number of samples that we want a leaf node to contain


Try - Random forest
 - params: n_estimators (120, 300, 500, 800), max_depth(5, 8, 15, 25, 30), min_samples_split(1, 2, 5, 10, 15, 100), min_samples_leaf(1, 2, 5, 10), max_features (log2, sqrt, None)

Try - SVM
 - params: C(.001, .01, 0.1, 1, 10, 100, 1000), gamma ('auto','scale'), class_weight ('balanced', None)

Try - KNN
 - params: n_neighbors (2, 4, 8, 16), p (2,3)

Then try all again with PCA


In [12]:
from sklearn.pipeline import Pipeline

In [ ]:
# Decision Tree Classifier
parameters = {'max_depth':[5, 8, 15, 25, 30], 'min_samples_split':[1, 2, 5, 10, 15, 100], 'min_samples_leaf':[1, 2, 5, 10]}
#clf = classifier
clf = GridSearchCV(LogisticRegression(solver='saga'), parameters, cv = 5)
clf.fit(X_train, y_train)